In [1]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path, encoding='utf-8'):
    # Read CSV file and convert it to a list of dictionaries
    with open(csv_file_path, 'r', encoding=encoding) as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = list(csv_reader)

    # Write the list of dictionaries to a JSON file
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage:
csv_file_path = 'Lyric_dataset.csv'  # Path to your CSV file
json_file_path = 'Lyric_dataset.json'  # Path to save the resulting JSON file
csv_to_json(csv_file_path, json_file_path)


In [2]:
import json

import chromadb

with open("Lyric_dataset.json", encoding='utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
#
# Create the database (in memory) and populate it.
#
# We assemble the text to be embedded like this:
documents = [
    f"The following content is about {item['artist_name']} and pertains to the track '{item['track_name']}' released in {item['release_date']} under the genre '{item['genre']}': {item['lyrics']}"
    for item in data
]
ids = [str(i) for i in range(len(documents))]
metadatas = [{k: v for k, v in item.items() if k != "lyrics"} for item in data]

chroma_client = chromadb.Client()

collection = chroma_client.create_collection(
    metadata={"hnsw:space": "cosine"}, name="Lyric_dataset"
)

collection.add(documents=documents, ids=ids, metadatas=metadatas)


def pretty_print(content):
    print(json.dumps(content, indent=2))

In [56]:
#
# Query by similarity.
#
query_text = "Can you make me a playlist with 70s and 80s southern rock music, sweet home alabama" 
n_results = 10
results = collection.query(n_results=n_results, query_texts=[query_text])
pretty_print(results)


{
  "ids": [
    [
      "8660",
      "9231",
      "16035",
      "9152",
      "9215",
      "9284",
      "16010",
      "9163",
      "9812",
      "9206"
    ]
  ],
  "distances": [
    [
      0.39068877696990967,
      0.3985518217086792,
      0.4122936725616455,
      0.42040205001831055,
      0.42590510845184326,
      0.43200987577438354,
      0.4336088299751282,
      0.43465250730514526,
      0.43619829416275024,
      0.4380617141723633
    ]
  ],
  "metadatas": [
    [
      {
        "acousticness": "0.181726086",
        "age": "0.657142857",
        "artist_name": "lynyrd skynyrd",
        "communication": "0.058784894",
        "danceability": "0.578685151",
        "dating": "0.061274947",
        "energy": "0.605593288",
        "family/gospel": "0.088386346",
        "family/spiritual": "0.029383546",
        "feelings": "0.021994503",
        "genre": "country",
        "instrumentalness": "0.00033502",
        "len": "92",
        "light/visual perceptions":

In [57]:
if 'metadatas' in results:
    for metadata_list in results['metadatas']:
        for metadata in metadata_list:
            track_name = metadata.get('track_name')
            artist_name = metadata.get('artist_name')
            if track_name and artist_name:
                print(f"Track: {track_name}, Artist: {artist_name}")

Track: sweet home alabama, Artist: lynyrd skynyrd
Track: mountain music, Artist: alabama
Track: dixie/sweet home alabama, Artist: lynyrd skynyrd
Track: gonna have a party, Artist: alabama
Track: if you're gonna play in texas (you gotta have a fiddle in the band), Artist: alabama
Track: fantasy, Artist: alabama
Track: sweet home alabama, Artist: the outlaws
Track: can't you see, Artist: alabama
Track: it's all comin' back to me now, Artist: alabama
Track: some other place, some other time, Artist: alabama
